In [22]:
import praw
import requests
import pandas as pd
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [23]:
REDDIT_CLIENT_ID = "suCjVen6MTRiU9E88cqRAg"
REDDIT_CLIENT_SECRET = "UiP81w6C1qyenfZzJ0asiegtJlQtZA"
REDDIT_USER_AGENT = "ECE473 Final Project"

# 🔹 Initialize Reddit API
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

# 🔹 Subreddits to Scrape
subreddits = ["NFT", "CryptoCurrency", "OpenSea", "NFTsMarketplace"]
# 🔹 Target NFT Collections for Sentiment Analysis
nft_collections = [
    "Bored Ape Yacht Club", "BAYC", "Mutant Ape Yacht Club", "MAYC",
    "Pudgy Penguins", "Azuki", "Doodles", "CloneX", "Moonbirds",
    "Otherdeed", "Milady", "CryptoPunks"
]


# 🔹 Function to Scrape Posts
def scrape_reddit_posts(limit=50):
    nft_posts = {collection: [] for collection in nft_collections}  # Store by collection

    for subreddit in subreddits:
        try:
            for post in reddit.subreddit(subreddit).new(limit=limit):
                for collection in nft_collections:
                    if collection.lower() in (post.title + " " + post.selftext).lower():
                        nft_posts[collection].append({
                            "title": post.title,
                            "text": post.selftext,
                            "url": post.url,
                            "score": post.score,
                            "created_utc": post.created_utc,
                            "subreddit": subreddit
                        })
            time.sleep(1)  # Prevent rate limits
        except Exception as e:
            print(f"Error scraping {subreddit}: {e}")

    return nft_posts  # Returns a dictionary of collections with their respective posts

# 🔹 Scrape and Save Data
nft_data = scrape_reddit_posts(limit=100)  # Get 100 posts per subreddit



In [24]:
# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment_by_collection(posts_dict):
    collection_sentiments = {}

    for collection, posts in posts_dict.items():
        if not posts:
            continue  # Skip if no mentions

        scores = [analyzer.polarity_scores(post["title"])["compound"] for post in posts]
        avg_sentiment = sum(scores) / len(scores) if scores else 0

        collection_sentiments[collection] = avg_sentiment

    return collection_sentiments

# Run sentiment analysis
nft_sentiments = analyze_sentiment_by_collection(nft_data)

# Show results
print(nft_sentiments)


{'Mutant Ape Yacht Club': 0.0, 'Pudgy Penguins': 0.0, 'Azuki': 0.0, 'Doodles': 0.3535, 'Milady': 0.0}


In [19]:
def get_nftperp_index_price(collection: str):
    """Fetches the index price of an NFT collection from NFTPerp."""
    url = f"https://api.nftperp.xyz/index-price/{collection}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data.get("indexPrice")
    else:
        print(f"Error fetching NFTPerp data: {response.text}")
        return None


In [28]:
def determine_trade_action(nft_collection, sentiment_score):
    """Determines whether to long or short an NFT collection based on sentiment score."""
    
    threshold = 0.05  # Adjust this based on testing
    
    if sentiment_score > threshold:
        return "LONG"
    elif sentiment_score < -threshold:
        return "SHORT"
    else:
        return "HOLD"

nft_data = scrape_reddit_posts(limit=100)  # Scrape posts per collection
nft_sentiments = analyze_sentiment_by_collection(nft_data)  # Get sentiment per collection

for collection, sentiment_score in nft_sentiments.items():
    trade_action = determine_trade_action(collection, sentiment_score)
    print(f"Trade decision for {collection}: {trade_action}")

Trade decision for Mutant Ape Yacht Club: HOLD
Trade decision for Pudgy Penguins: HOLD
Trade decision for Azuki: HOLD
Trade decision for Doodles: LONG
Trade decision for Milady: HOLD
